<a href="https://colab.research.google.com/github/mogesTesema/Machine-Learning-Mastery-With-TensorFlow/blob/main/05_transfer_learning_in_tensorflow_part_2_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning with tensorflow part 2: Fine-Tuning

## Creating helper functions

In [1]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py

--2025-10-18 14:50:46--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2025-10-18 14:50:47 (94.3 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [2]:
from helper_functions import plot_loss_curves,create_tensorboard_callback,confusion_matrix,load_and_prep_image,walk_through_dir,unzip_data

## Let's get some data

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

--2025-10-18 14:50:52--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.152.207, 142.250.125.207, 192.178.210.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.152.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   239MB/s    in 0.7s    

2025-10-18 14:50:53 (239 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [4]:
unzip_data('10_food_classes_10_percent.zip')

In [5]:
walk_through_dir("10_food_classes_10_percent")

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent/test'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/pizza'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/fried_rice'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_curry'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/grilled_salmon'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/hamburger'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ramen'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/steak'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_wings'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ice_cream'.
There are 0 directories and 250 images in '10_food_classes_10_percent

In [6]:
# Create training and test directory paths
train_dir = "/content/10_food_classes_10_percent/train"
test_dir = "/content/10_food_classes_10_percent/test"


In [7]:
import tensorflow as tf

In [8]:
IMG_SIZE = (224,224)
BATCH_SIZE = 32
train_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                            image_size=IMG_SIZE,
                                                                            label_mode='categorical',
                                                                            batch_size=BATCH_SIZE
                                                                            )
test_data = tf.keras.preprocessing.image_dataset_from_directory(directory=test_dir,
                                                                image_size=IMG_SIZE,
                                                                batch_size=BATCH_SIZE,
                                                                label_mode="categorical")


Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [9]:
train_data_10_percent

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))>

In [10]:
train_data_10_percent.class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [11]:
for images,labels in train_data_10_percent.take(1):
  print(images,labels)

tf.Tensor(
[[[[1.49056122e+02 1.43056122e+02 1.31056122e+02]
   [1.56000015e+02 1.50000015e+02 1.38000015e+02]
   [1.60290817e+02 1.54290817e+02 1.40852036e+02]
   ...
   [1.30566330e+02 1.25566338e+02 1.03566338e+02]
   [1.31571411e+02 1.26571419e+02 1.06571419e+02]
   [1.30285721e+02 1.25285713e+02 1.05285713e+02]]

  [[1.55357147e+02 1.52357147e+02 1.37357147e+02]
   [1.59637756e+02 1.56637756e+02 1.41637756e+02]
   [1.56515305e+02 1.53515305e+02 1.38117355e+02]
   ...
   [1.27341812e+02 1.25341812e+02 1.02341812e+02]
   [1.27209198e+02 1.25209198e+02 1.02209198e+02]
   [1.29403091e+02 1.27403091e+02 1.04403091e+02]]

  [[1.61290817e+02 1.58290817e+02 1.40862244e+02]
   [1.57586731e+02 1.54586731e+02 1.37158157e+02]
   [1.50188782e+02 1.47188782e+02 1.29760208e+02]
   ...
   [1.28954086e+02 1.26954086e+02 1.04382660e+02]
   [1.30285721e+02 1.28285721e+02 1.05714302e+02]
   [1.32000000e+02 1.30000000e+02 1.07428574e+02]]

  ...

  [[2.75815506e+01 7.64283514e+00 1.65300179e+00]
   [2

In [12]:
resNet_model = tf.keras.applications.ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3))


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [13]:
resNet_weights = resNet_model.get_weights()




In [14]:
resNet_model.compile(loss='categorical_crossentropy',
                     optimizer='adam',
                     metrics=['accuracy'])
# resNet_model.fit(train_data_10_percent,epochs=3)

## Creating the model using functional API.
what is functional API actually

In [15]:
def getMe():
  def hello(name):
    return f"hello {name} functional API"
  return hello

result = getMe()("Keras")
print(result)

hello Keras functional API


## Modle 0: Building a transfer learning model using the Keras Functional API



In [ ]:
# 1. Create the base model with tf.keras.applications.resNet50
base_model = tf.keras.applications.EfficientNetB0(include_top=False)

# 2. Freeze the base model (so the underling pre-trained patterns aren't updated durring training)
base_model.trainable = False

# 3. Create inputs into our model
inputs = tf.keras.layers.Input(shape=(224,224,3), name="input_layer")

# 4. If you using a model like ResNet50V2 you will need to normalize inputs
# x = tf.keras.layers.experimental.preprocessing.Rescaling(1/255.)(inputs)

# 5. Pass the inputs to the base_model
x = base_model(inputs)
print(f"Shape after passing inputs through base model: {x.shape}")

# 6. Average pool the outputs of the base model (aggregate all the the most important information reduce number of computations)
x = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)
print(f"Shape after global average 2D: {x.shape}")

# 7. Create the output activation layer
outputs = tf.keras.layers.Dense(10,activation="softmax",name="output_layer")(x)

# 8. Combine the inputs with the outputs into a model
model_0 = tf.keras.Model(inputs,outputs)

# 9. compile the model
model_0.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# 10. Fit the model and save its history
model_0_history = model_0.fit(train_data_10_percent,
                              epochs=5,
                              steps_per_epoch=len(train_data_10_percent),
                              validation_data= test_data,
                              validation_steps= len(test_data),
                              callbacks=[create_tensorboard_callback("TensorFlow_Hub","Fine_tuned_EfficientNet")]
                              )




16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Shape after passing inputs through base model: (None, 7, 7, 1280)
Shape after global average 2D: (None, 1280)
Saving TensorBoard log files to: TensorFlow_Hub/Fine_tuned_EfficientNet/20251018-145100
Epoch 1/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2262 - loss: 2.1878

In [ ]:
plot_loss_curves(model_0_history)

In [ ]:
# Evaluate the tuned model
model_0.evaluate(test_data)

In [ ]:
for layer_number, layer in enumerate(base_model.layers):
  print(layer_number,layer.name)

In [ ]:
base_model.summary()

In [ ]:
# tf.keras.utils.plot_model(base_model)

In [ ]:
model_0.summary()

## Getting a feature vector from a trained model

In [ ]:
# # Define the input shape
# input_shape = (1,4,4,3)
# # Create a random tensor
# tf.random.set_seed(42)
# input_tensor = tf.random.normal(input_shape)
# # pass the random tensor through a global average pooling 2D layer
# global_average_pooled_tensor = tf.keras.layers.GlobalAveragePooling2D(input_tensor)
# global_average_pooled_tensor

## Running a series of transfer learning experiments
we've seen the incredible results transfer learning can get with only 10% of the training data, but how does it go with 1% of the training data.. how about we set up a bunch of experiments to find out:
1. `model_1`  - use feature extracion transfer learning with 1% of the training data with data augmentation
2. `model_2` - use feature extracctio transfer learning with 10% of the training data with data augmentation
3. `model_3` - use fine-tuning transfer learning on 10% of the training data with data augmentation
4. `model_4` - use fine-turning transfer learning on 100% of the training data with data augmentation
**Note:** throught all experiments the same test dataset will be used to evaluate our model.. this ensures consisency across evaluation metrics.


In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_1_percent.zip

In [ ]:
unzip_data("10_food_classes_1_percent.zip")

In [ ]:
walk_through_dir("10_food_classes_1_percent")

In [ ]:
train_dir_1_percent = "10_food_classes_1_percent/train"
test_dir = "10_food_classes_1_percent/test"


In [ ]:
# Set up data loader
IMG_SIZE = (224,224)
BATCH_SIZE = 32
train_data_1_percent = tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir_1_percent,
                                                             image_size=IMG_SIZE,
                                                             label_mode="categorical",
                                                             batch_size=BATCH_SIZE,
                                                             )
test_data = tf.keras.preprocessing.image_dataset_from_directory(directory=test_dir,
                                                  image_size=IMG_SIZE,
                                                  batch_size=BATCH_SIZE,
                                                  label_mode="categorical",
                                                  )

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import preprocessing


In [ ]:
data_augmentation = tf.keras.Sequential([
    # tf.keras.layers.Rescaling(1/255.), # EfficientNet has rescaling built-in
    # tf.keras.layers.Input(shape=(224,224,3)),
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomHeight(0.2),
    tf.keras.layers.RandomWidth(0.2),
    # tf.keras.layers.RandomErasing(factor=1,scale=(0.02,0.03)),
    tf.keras.layers.RandomZoom(height_factor=(0.1,0.15)),
    ],name="data_augmentation")

In [ ]:
# view random image and compare with original image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import random
target_class = random.choice(train_data_1_percent.class_names)
target_dir = "10_food_classes_1_percent/train/" + target_class
random_image = random.choice(os.listdir(target_dir))
random_image_path = os.path.join(target_dir,random_image)
img = mpimg.imread(random_image_path)
plt.imshow(img)
plt.title(f"original random image from class: {target_class}")
plt.axis(False)
plt.figure();
augmented_imag = data_augmentation(img)
print(augmented_imag.shape)
plt.imshow(augmented_imag/255.)
plt.title(f"augmented random image from class: {target_class}")
plt.axis(False)


## Model 1: feature extraction transfer learning with 1% of the data with data augmentation

In [ ]:
#data augmentation with functional layers
# Recommended way for TF ≥ 2.17
inputs = tf.keras.Input(shape=(224, 224, 3), name="input_layer")

# Apply augmentation functionally (not in a Sequential)
x = tf.keras.layers.RandomFlip("horizontal")(inputs)
x = tf.keras.layers.RandomRotation(0.2)(x)
x = tf.keras.layers.RandomHeight(0.2)(x)
x = tf.keras.layers.RandomWidth(0.2)(x)
x = tf.keras.layers.RandomZoom(0.1, 0.15)(x)
# Setup input shape and base_model
input_shape = (224,224,3)
base_model = tf.keras.applications.EfficientNetB0(include_top=False,input_shape=(224,224,3))
base_model.trainable = False

# Create the input layer
# inputs = layers.Input(shape=input_shape,name="input_layer")

# add in data augmentation Sequential model as a layer
# x = data_augmentation(inputs)

# Give base_model the inputs( after augmentation) and don't train it
x = base_model(x,training=False)

# pool output features of the base model
x = layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)

# put a dense layer on as the output
outputs = layers.Dense(10,activation="softmax",name="output_layer")(x)

# Make a model using the inputs and outputs
model_1 = keras.Model(inputs=inputs,outputs=outputs)

# compile the model
model_1.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
# fit the model
model_1_history = model_1.fit(train_data_1_percent,
                              epochs=5,
                              steps_per_epoch=len(train_data_1_percent),
                              validation_data=test_data,
                              validation_steps=(int(0.25*len(test_data))),
                              callbacks=[create_tensorboard_callback("TensorFlow_Hub","1_percent_data_exprt")]
                              )










In [ ]:
plot_loss_curves(model_1_history)

In [ ]:
model_1.summary()

In [ ]:
# Evaluate with test dataset
model_1.evaluate(test_data)

## Model 2: feature extraction transfer learning model with 10% of data and data augmentation

In [ ]:
# Get 10% of data
train_dir_10_percent = "/content/10_food_classes_10_percent/train"
test_dir = "/content/10_food_classes_10_percent/test"


In [ ]:
# Set data inputs
IMG_SIZE = (224,224)
train_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(train_dir_10_percent,
                                                                            label_mode="categorical",
                                                                            image_size=IMG_SIZE
                                                                            )
test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode="categorical",
                                                                image_size=IMG_SIZE)

In [ ]:
# Create model 2 with data augmentation built in
from tensorflow.keras import layers
inputs = layers.Input(shape=(224,224,3),name="input_layer")
x = layers.RandomFlip("horizontal")(inputs)
x = layers.RandomHeight(0.2)(x)
x = layers.RandomWidth(0.2)(x)
x = layers.RandomZoom(0.2)(x)
x = layers.RandomRotation(0.2)

# setup the input shape to our model
input_shape = (224,224,3)
# create a frozen base model
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False
x = base_model(x,trainabl=False)
x = layers.GlobalAveragePooling2D(name="Global_average_pooling_layer")(x)
outputs =  layers.Dense(10,activation="softmax",name="output_layer")(x)
model_2 = tf.keras.Model(inputs,outputs)

# compile the model
model_2.compile(loss="categorical_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])

# Fit the model
model_2_history = model_2.fit(train_data_10_percent,
                              epochs=5,
                              steps_per_epoch=len(train_data_10_percent),
                              validation_data=test_data,
                              validation_steps=int(0.25*len(test_data)))

